In [1]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
import torch
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

In [2]:
import psutil
import torch
import os
import multiprocessing

print('Total memory:', psutil.virtual_memory().total / (1024 ** 3), "GB")
print("Number of CPUs:", multiprocessing.cpu_count())
torch.set_num_threads(multiprocessing.cpu_count())

print("PyTorch is using", torch.get_num_threads(), "threads.")

Total memory: 93.06243133544922 GB
Number of CPUs: 64
PyTorch is using 64 threads.


# Dataset 1

In [3]:
data = pd.read_csv('filtered_data.csv')
sequences = list(data['seq'])

In [4]:
max_length = max(len(seq) for seq in sequences)

# Regulon Dataset

In [5]:
data = pd.read_csv('promoter_processed.csv')
sequences = list(data['Sequences'])

In [6]:
max_length = max(len(seq) for seq in sequences)
max_length

82

## Embedding Generation Function

In [10]:
#if you're running on a GPU:

print(torch.cuda.is_available())

def generate_embeddings(model_name):

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForMaskedLM.from_pretrained(model_name, trust_remote_code=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    sequences_short = sequences  
    batch_size = 64 
    batches = [sequences_short[i:i + batch_size] for i in range(0, len(sequences_short), batch_size)]

    all_embeddings = []
    print("gonna start")

    with torch.no_grad(): 
        for batch in tqdm(batches, desc="Processing sequences"):
            tokens_ids = tokenizer.batch_encode_plus(batch, return_tensors="pt", padding="max_length", max_length=max_length)["input_ids"]
            attention_mask = tokens_ids != tokenizer.pad_token_id

            #move tensors to GPU
            tokens_ids = tokens_ids.to(device)
            attention_mask = attention_mask.to(device)

            #model forward pass
            torch_outs = model(
                tokens_ids,
                attention_mask=attention_mask,
                encoder_attention_mask=attention_mask,
                output_hidden_states=True
            )

            embeddings = torch_outs['hidden_states'][-1]
            attention_mask = torch.unsqueeze(attention_mask, dim=-1)
            masked_embeddings = attention_mask * embeddings
            
            max_sequence_embeddings, _ = torch.max(masked_embeddings, dim=-2)

            all_embeddings.append(max_sequence_embeddings)

    #move the final result to CPU and convert to NumPy array
    all_embeddings = torch.cat(all_embeddings, dim=0).cpu()

    print(f"All Embeddings shape for {model_name}: {all_embeddings.shape}")
    
    return all_embeddings
    
all_embeddings_500m = generate_embeddings("InstaDeepAI/nucleotide-transformer-500m-1000g")
# all_embeddings_dnab = generate_embeddings("zhihan1996/DNABERT-2-117M")

True


gonna start


Processing sequences: 100%|██████████| 104/104 [01:18<00:00,  1.33it/s]


All Embeddings shape for InstaDeepAI/nucleotide-transformer-500m-1000g: torch.Size([6646, 1280])


In [ ]:
all_embeddings_500m_max = generate_embeddings("InstaDeepAI/nucleotide-transformer-500m-1000g")

gonna start


Processing sequences:  63%|██████▎   | 66/104 [00:50<00:30,  1.26it/s]

In [12]:
#saving to a npy file:
import pickle

np.save('all_embeddings_100m_max', all_embeddings_500m_max)


# with open('embeddings_500m_human.pkl', 'wb') as f:
#     pickle.dump(all_embeddings, f)

pickle_file_size = os.path.getsize('all_embeddings_100m_max.npy')

pickle_file_size

34027648

In [ ]:
#CPU Implementation

tokenizer = AutoTokenizer.from_pretrained("InstaDeepAI/nucleotide-transformer-v2-50m-multi-species", trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained("InstaDeepAI/nucleotide-transformer-v2-50m-multi-species", trust_remote_code=True)

sequences_short = sequences
batch_size = 48 
batches = [sequences_short[i:i + batch_size] for i in range(0, len(sequences_short), batch_size)]

all_embeddings = []

for batch in tqdm(batches, desc="Processing sequences"):
    tokens_ids = tokenizer.batch_encode_plus(batch, return_tensors="pt", padding="max_length", max_length=max_length)["input_ids"]
    attention_mask = tokens_ids != tokenizer.pad_token_id

    torch_outs = model(
        tokens_ids,
        attention_mask=attention_mask,
        encoder_attention_mask=attention_mask,
        output_hidden_states=True
    )

    embeddings = torch_outs['hidden_states'][-1].detach().numpy()
    attention_mask = torch.unsqueeze(attention_mask, dim=-1)
    mean_sequence_embeddings = torch.sum(attention_mask * embeddings, axis=-2) / torch.sum(attention_mask, axis=1)

    all_embeddings.append(mean_sequence_embeddings)

all_embeddings = torch.cat(all_embeddings, dim=0)

print(f"All Embeddings shape: {all_embeddings.shape}")

In [ ]:
len(all_embeddings)